In [3]:

# Download Data from Yahoo Finance
import datetime as dt
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
# Pour lire les données de Yahoo Finance, vous devez installer le toolbox yfinance
import yfinance as yf 

def get_dow():
    start_sp = dt.datetime(2004,12,31)
    end_sp = dt.datetime(2019, 12,31)

    # Dow Jones index
    # 1st Dow Jones Index + 30 components
    dow30 = ['^DJI','AXP', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'DD', 'XOM',
            'GE', 'GS', 'HD', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM',
            'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PFE', 'PG',
            'TRV', 'UNH', 'UTX', 'VZ', 'V', 'WMT', 'DIS']

    dow_data = yf.download(dow30, start=start_sp, end=end_sp,group_by="ticker")

    return dow_data

In [139]:
def calculate_fivedays_roi(stock_price, date):
    roi = None
    try:
        past_date = date - np.timedelta64(5, "D")

        current_value = stock_price.loc[date]
        past_value = stock_price.loc[past_date]

        roi = (current_value - past_value)/past_value * 100
        
        
    except KeyError:
        
    return roi



# var historique sur periode de 250 jours et comparaison 5 jours avants
def historic_var(companies, date):
    period = 250
    
    # get time interval
    end_date = np.datetime64(date)
    start_date = end_date - np.timedelta64(period+5, "D")
    #download le prix du stock des compagnies sur l'intervalle de 255 jours (250 jours de rendemment nécessitant 5 jours de recul)
    stock_prices = yf.download(companies, start=start_date, end=end_date,group_by="ticker")
    stock_prices = stock_prices.transpose() 
    
    result = pd.DataFrame()
    # for each stock in the portfolio
    for company in companies:
        print("Calculating ROI for {0}".format(company))
        rois = []
        dates = []
        company_stock_price = stock_prices.loc[(company, "Close")]
        
        # calculate de ROI over the last 250 days
        curr_date = end_date - np.timedelta64(period, "D")
        while(curr_date < end_date):
            roi = calculate_fivedays_roi(company_stock_price, curr_date)
            if roi:
                rois.append(roi)
                dates.append(curr_date)
            curr_date = curr_date + np.timedelta64(1, "D")                
            
        result[company] = pd.Series(rois,index=dates)
    return result
    
    

IndentationError: expected an indented block (<ipython-input-139-aa07628eb8dd>, line 14)

In [140]:
stocks = ['DJI','AXP', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'DD', 'XOM',
            'GE', 'GS', 'HD', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM',
            'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PFE', 'PG',
            'TRV', 'UNH', 'UTX', 'VZ', 'V', 'WMT', 'DIS']
a = historic_var(stocks,"2019-03-01")


[*********************100%***********************]  31 of 31 completed
Calculating ROI for DJI


UnboundLocalError: local variable 'test' referenced before assignment

In [112]:
a

,DJI,AXP,AAPL,BA,CAT,CSCO,CVX,DD,XOM,GE,...,NKE,PFE,PG,TRV,UNH,UTX,VZ,V,WMT,DIS
2019-03-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-03-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-03-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-03-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-03-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-03-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-03-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-03-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-03-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
